# Import Libraries and Load Accelerometer-Gyroscope Dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc, roc_curve
from sklearn.preprocessing import RobustScaler
import time
import pickle
import os

import warnings
warnings.filterwarnings('ignore')

columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']
df = pd.read_csv("Scaled_Combined_Active_Gyro_Accel_Cleaned.csv", usecols=columns)

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

# Display Preprocessed Sensor Data

In [2]:
df

,Accel X (g),Accel Y (g),Accel Z (g),Gyro X (°/s),Gyro Y (°/s),Gyro Z (°/s),Activity_Label
0,-5.983110,1.247379,8.365341,0.321315,-0.024435,-0.131947,0
1,-5.983110,1.247379,8.365341,0.321315,-0.024435,-0.131947,0
2,-5.983110,1.247379,8.365341,0.321315,-0.024435,-0.131947,0
3,-5.983110,1.247379,8.365341,0.321315,-0.024435,-0.131947,0
4,-5.983110,1.247379,8.365341,0.321315,-0.024435,-0.131947,0
...,...,...,...,...,...,...,...
230160,8.446744,-4.716577,3.725378,-0.808786,0.274889,-0.142942,9
230161,8.446744,-4.716577,3.725378,-0.808786,0.274889,-0.142942,9
230162,8.446744,-4.716577,3.725378,-0.808786,0.274889,-0.142942,9
230163,8.446744,-4.716577,3.725378,-0.808786,0.274889,-0.142942,9


# Define Predictor and Target Variables

In [3]:
PredictorCol = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
                'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)']
TargetCol = 'Activity_Label'

X = df[PredictorCol].values
y = df[TargetCol].values

# Split Sensor Data into Training and Testing Sets (Stratified)

In [4]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X, y, test_size=0.30, 
                                                    random_state=15, 
                                                    stratify = y)

# Build, Train, and Evaluate LSTM Model

In [5]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

X_train_sm = np.reshape(X_train_sm, (X_train_sm.shape[0], 1, X_train_sm.shape[1]))
X_test_sm = np.reshape(X_test_sm, (X_test_sm.shape[0], 1, X_test_sm.shape[1]))

model = Sequential()
model.add(LSTM(128, input_shape=(1, X_train_sm.shape[2]), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(16, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_sm, y_train_sm, epochs=10, batch_size=32)

test_loss, test_accuracy = model.evaluate(X_test_sm, y_test_sm)

print("Test Accuracy:", test_accuracy)

2025-05-07 14:23:03.009811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-07 14:23:03.019048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-07 14:23:03.021863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 14:23:03.029567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-07 14:23:03.613503: W tensorflow/compiler/tf2

Epoch 1/10


2025-05-07 14:23:06.233773: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


5035/5035 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5311 - loss: 1.2714
Epoch 2/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.8916 - loss: 0.3270
Epoch 3/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9709 - loss: 0.1056
Epoch 4/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9833 - loss: 0.0557
Epoch 5/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9885 - loss: 0.0366
Epoch 6/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9902 - loss: 0.0316
Epoch 7/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.9925 - loss: 0.0254
Epoch 8/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9934 - loss: 0.0243
Epoch 9/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9955 - loss: 0.0167
Epoch 10/10
5035/5035 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9956 - loss: 0.0169
2158/2158 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9987 - loss: 0.0090
Test Accuracy: 0.9988124370574951


# Save LSTM Model in Keras Format

In [6]:
model.save("model.keras")

# Save LSTM Model Architecture and Weights Separately

In [7]:
import pickle

with open('LSTMmodel_architecture.pkl', 'wb') as f:
    pickle.dump(model, f)
print("LSTM model architecture has been successfully saved to 'LSTMmodel_architecture.pkl'")

with open('LSTMmodel_weights.pkl', 'wb') as f:
    pickle.dump(model.get_weights(), f)
print("LSTM model weights have been successfully saved to 'LSTMmodel_weights.pkl'")

LSTM model architecture has been successfully saved to 'LSTMmodel_architecture.pkl'
LSTM model weights have been successfully saved to 'LSTMmodel_weights.pkl'
